In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import cv2 as cv

In [5]:
df=pd.read_csv("train.csv")
df.head

<bound method NDFrame.head of        label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0          1       0       0       0       0       0       0       0       0   
1          0       0       0       0       0       0       0       0       0   
2          1       0       0       0       0       0       0       0       0   
3          4       0       0       0       0       0       0       0       0   
4          0       0       0       0       0       0       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
41995      0       0       0       0       0       0       0       0       0   
41996      1       0       0       0       0       0       0       0       0   
41997      7       0       0       0       0       0       0       0       0   
41998      6       0       0       0       0       0       0       0       0   
41999      9       0       0       0       0       0       0       0       0   

       pi

In [6]:
ydata=df.label
ydata

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [7]:
xdata=df.drop('label', axis='columns')
xdata=xdata/255

In [8]:
xtrain,xtest,ytrain,ytest=train_test_split(xdata,ydata,test_size=0.25)

In [9]:
model=KNeighborsClassifier(n_neighbors=180)

In [10]:
model.fit(xtrain,ytrain)

KNeighborsClassifier(n_neighbors=180)

In [11]:
model.score(xtest,ytest)

0.9081904761904762

In [12]:

threshold = 150

def get_threshold(val):
    global threshold
    threshold = val

def display(model):
    vid = cv.VideoCapture(0)  
    cv.namedWindow('digit_detection')
    cv.createTrackbar('threshold','digit_detection',150, 255,get_threshold)
    digit_detection = np.zeros((500, 500), np.uint8)
    framecount = 0
    while True:
        res,frame=vid.read()
        framecount+=1
        grayframe=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
        _,thr=cv.threshold(grayframe, threshold,255,cv.THRESH_BINARY_INV)
        resizedframe=thr[250-100:250+100, 250-100:250+100]
        digit_detection[250-100:250+100, 250-100:250+100] =resizedframe
        testimg=cv.resize(resizedframe,(28, 28))
        testimg=testimg.reshape(1,-1)/255
        testimg_df=pd.DataFrame(testimg, columns=model.feature_names_in_)
        res = str(model.predict(testimg_df)[0])
        if framecount==5:
            digit_detection[0:500,0:80]=0
            framecount=0
        cv.putText(digit_detection,res,(10, 80),cv.FONT_HERSHEY_SIMPLEX,2,(255,255,255),3)
        cv.rectangle(digit_detection,(250-105, 250-105),(250+105, 250+105),(255, 255, 255),thickness=3)
        cv.imshow('digit_detection',digit_detection)
        if cv.waitKey(1)&0xFF ==ord('q'):
            break

    #for realising the resources
    vid.release()
    cv.destroyAllWindows()


In [13]:
display(model)